# Lightgbm with grid search for hyperparameter optimisation

In this section we are going to create a boosting model for our classification problem.
Boosting is an algorithm which set out to answer the question "Can a set of weak learners create a single strong learner?"
It turns out to be very successful in a wide array of applications.

Lightgbm, short for light gradient-boosting machine, is a specific boosting framework developed by microsoft and released open source in 2016.
Although less widely used than XGboost, lightgbm has advantages in efficiency and memory consumption.

I originally wished to use XGboost, but due to some of the problems we came across when implementing the model, lightgbm was the better choice.

In [5]:
import numpy as np
import pandas as pd
from datetime import datetime
from numba import jit
import lightgbm as lgbm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

Here we have our functions to calculate the gini coefficient, and implement the data handling code to sort out missing values / drop the columns which are mostly missing values and also the calc columns since our EDA discovered these had no correlation to the target. Furthermore, we encode our catagorical features and rescale the data.

In [6]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds' % (thour, tmin, round(tsec, 2)))


@jit
def gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n - 1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini


def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', gini(labels, preds), True


def dropmissingcol(pdData):
    vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
    pdData.drop(vars_to_drop, inplace=True, axis=1)
    return pdData


def missingvalues(pdData):
    mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
    mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')
    features = ['ps_reg_03', 'ps_car_12', 'ps_car_14', 'ps_car_11']
    for i in features:
        if i == 'ps_car_11':
            pdData[i] = mode_imp.fit_transform(pdData[[i]]).ravel()
        else:
            pdData[i] = mean_imp.fit_transform(pdData[[i]]).ravel()
    return pdData


def encodecat(train, test):
    cat_features = [col for col in train.columns if '_cat' in col]
    for column in cat_features:
        temp = pd.get_dummies(pd.Series(train[column]), prefix=column)
        train = pd.concat([train, temp], axis=1)
        train = train.drop([column], axis=1)

    for column in cat_features:
        temp = pd.get_dummies(pd.Series(test[column]), prefix=column)
        test = pd.concat([test, temp], axis=1)
        test = test.drop([column], axis=1)
    return train, test


def RescaleData(train, test):
    scaler = StandardScaler()
    scaler.fit_transform(train)
    scaler.fit_transform(test)
    return train, test


def DropCalcCol(train, test):
    col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
    train = train.drop(col_to_drop, axis=1)
    test = test.drop(col_to_drop, axis=1)
    return train, test

We begin by loading our data - This is the data where the missing values have already been imputed by a linear regression model.
We can later set our impute boolean to False and compare how effective this model was in comparison to a simple mean/mode imputation.

Applying the functions above, we have an encoded, rescaled dataset with missing values imputed. The targets have been seperated as new dataframes.

In general, it is not necessary to scale or encode your data when using boosting algorithms. Boosting algorithms typically work by building a model based on a combination of many weak models, each of which is trained on a subset of the data. This means that boosting algorithms are less sensitive to the scale of the data than some other types of algorithms, such as support vector machines. However, it is unlikely to lower the performance of the algorithm and in some cases could result in increased performance. In this example, it has been done since the functions have already been written for our previous models.

In [7]:
Kaggle = False #true if we want to use the full dataset which we submit to kaggle leaderboards
impute = True #true if we use linear regression to impute missing values - false uses mean/mode imputation
if Kaggle == False:
    if impute == True:
        train = pd.read_csv("Dataset/imputeTrain.csv")
        test = pd.read_csv("Dataset/imputetest.csv")
    else:
        train = pd.read_csv("Dataset/new_train.csv")
        test = pd.read_csv("Dataset/new_test.csv")
        test = dropmissingcol(test)
        train = dropmissingcol(train)
    target_test = test['target'].values
    test = test.drop(['target'], axis=1)
else:
    if impute == True:
        train = pd.read_csv("Dataset/imputetrainKag.csv")
        test = pd.read_csv("Dataset/imputetestKag.csv")
    else:
        train = pd.read_csv("Dataset/train.csv")
        test = pd.read_csv("Dataset/test.csv")
        test = dropmissingcol(test)
        train = dropmissingcol(train)

#code to clean up any remaining missing values with mean impute
train = missingvalues(train)
test = missingvalues(test)

#removing these columns from the dataframes and saving them seperately
y_train = train['target'].values
train_id = train['id'].values
X = train.drop(['target', 'id'], axis=1)
test_id = test['id']
X_test = test.drop(['id'], axis=1)

#encoding, rescaling and dropping calc columns
X, X_test = DropCalcCol(X, X_test)
X, X_test = encodecat(X, X_test)
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)
X, X_test = RescaleData(X, X_test)

We are now ready to implement the model.

XGboost vs lightgbm

Originally we set out on the project to implement an XGboost algorithm. The reasoning behind this was due to the fact that they often perform extremely well in tasks similar to the one here, especially when looking at the Kaggle leaderboards (albeit prone to overfitting)
When implementing a boosting algorithm, a huge part of the success comes from parameter and hyperparameter optimisation.
A method we have previously looked at is Grid Search CV for hyperparameter optimisation - effectively brute searching through a collection of potential hyperparameter combinations and returning the best result. An issue with this method is that the more precision you want, the more combinations and possibilities you will have to try.

An XGboost algorithm simply fell short on this big dataset as it was going to take a long time to run a grid search.
Possibilities were to reduce the dataset size for the grid search - say run the grid search on 20% of the data,
Or research into Lightgbm, an alternative boosting method with solid claims of being a lot more efficient in run time.

When implementing lightgbm the difference was huge. We could maintain the same full dataset and search through a large amount of hyperparameter combinations to optimise, which resulted in a noticeable score increase in comparison to our xgboost algorithm with a small hyperparameter search grid.

Therefore, for this project, we found lightgbm to be a better choice of algorithm.

In [8]:
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50

#paramaters to search over
params = {
    'min_child_weight': [5, 10, 12, 15, 30, 50, 100, 150],
    'num_leaves': [4, 5, 8, 10, 15, 20, 30],
    'subsample': [0.2, 0.4, 0.6, 0.8],
    'drop_rate': [0.1, 0.3, 0.5, 0.7, 0.15, 0.2],
    'max_depth': [3, 4, 5, 7, 10, 12, 15, 20]
}
#classifier model
model = lgbm.LGBMClassifier(learning_rate=LEARNING_RATE, n_estimators=600, objective='binary', )

#folds to use in stratified k-fold
folds = 3
#how many combinations of the above parameters should we try
param_comb = 10
#the algorithm is going to run folds x param_comb times

SKfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1)
#set up search with SKfold split
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4,
                                   cv=SKfold.split(X, y_train), verbose=3, random_state=1)

start_time = timer(None)
#start search
random_search.fit(X, y_train)
timer(start_time)

print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best Normalised gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('lightgbm-randomgridsearch-results-03.csv')

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  3.6min finished



 Time taken: 0 hours 3 minutes and 42.08 seconds

 All results:
{'mean_fit_time': array([21.58737771, 19.12638966, 15.78693207, 15.82215468, 21.41994246,
       21.02802483, 16.43050567, 21.41161577, 20.29587475, 17.21015843]), 'std_fit_time': array([0.2613967 , 3.07159906, 0.17857145, 0.11671209, 0.37230656,
       0.52583291, 0.27600376, 0.38582541, 0.27645544, 1.23320359]), 'mean_score_time': array([5.73831407, 6.08877707, 5.25174658, 5.53009836, 8.09494599,
       8.65806071, 5.74578905, 9.86506971, 9.85728606, 6.55046264]), 'std_score_time': array([0.07279661, 0.06160757, 0.09496823, 0.03288312, 0.27915329,
       0.35669044, 0.0378694 , 0.10645822, 0.29423084, 0.57578333]), 'param_subsample': masked_array(data=[0.8, 0.2, 0.4, 0.4, 0.8, 0.2, 0.2, 0.4, 0.8, 0.6],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_num_leaves': masked_array(data=[8, 10, 8, 4, 8, 15, 5, 

In grid search, a set of possible values for each hyperparameter is defined, and the combination of these values forms a grid. The grid search algorithm will then train the model with each combination of hyperparameters and evaluate the performance of the model on a validation set. The combination of hyperparameters that produces the best performance on the validation set is chosen as the best set of hyperparameters for the model.

Grid search can be computationally expensive, as it requires training the model multiple times with different combinations of hyperparameters. However, it is a simple and effective method for finding the best hyperparameters for a given model and dataset.

Here we have set up a grid search with the hyperparameters of interest to search over. The other parameters we can make a good guess from lightgbm literature online or they lack importance to fine tune in this particular case.

In this notebook, we have set to 3 folds and 10 combinations only to make the notebook accessible.

We run the grid search over a k=5 StratifiedKfold and search through with n_iter as the amount of combinations we wish to look at.
This is still a time intensive exercise, it is run in parallel across 4 chains but the lightgbm model has to train on five folds for each combination, and this is about 20-30 seconds each. For the strongest parameters I will use in the final model, we searched 200 combinations taking around 2 hours. This is overkill

In the future, it would be worthwhile to look into bayesian optimisation for hyperparameters. This approach will solve time problems as well as getting a more exact result because it allows the algorithm to focus on the most promising combinations of hyperparameter values.

In [9]:
'''
Best hyperparameters from grid search:
{'subsample': 0.2, 'num_leaves': 15, 'min_child_weight': 150, 'max_depth': 3, 'drop_rate': 0.15}
'''

"\nBest hyperparameters from grid search:\n{'subsample': 0.2, 'num_leaves': 15, 'min_child_weight': 150, 'max_depth': 3, 'drop_rate': 0.15}\n"

Now we have found our best parameters, we are ready to train the model and predict on the test set

When working with a dataset that has a class imbalance, stratified k-fold can be especially useful. This is because a dataset with a class imbalance can cause the model to be biased towards the majority class, and the evaluation of the model may be misleading if the folds are not representative of the class distribution in the dataset. By using stratified k-fold cross-validation, the model can be trained and evaluated on balanced folds, which can provide more accurate estimates of the model's performance.

We also wish to acknowledge the issue of overfitting, so we set early stopping times and iterate this process of generating our predictions by averaging over different folds and furthermore averaging the whole process over different seeds. Taking only our best folds would overfit here. Using multiple models trained with different random seeds will have slightly different parameter values and will make slightly different predictions. Averaging the predictions of these models can help smooth out any overfitting that may have occurred in individual models,

In [10]:
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50

#use our best parameters
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": LEARNING_RATE,
          "max_bin": 256,
          "n_estimators": 600,
          "verbosity": -1,
          "feature_fraction": feature_fraction,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_split_gain": 0,
          'subsample': 0.2,
          'num_leaves': 15,
          'min_child_weight': 150,
          'max_depth': 3,
          'drop_rate': 0.15
          }

folds = 5

SKfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1)

#empty, will save our scores here in the future
best_trees = []
fold_scores = []

#cv_train = np.zeros(len(y_train))
cv_pred = np.zeros(len(X_test))

start_time = timer(None)
#iterations each have a different seed, we average over these to prevent overfit
iterations = 3
for seed in range(iterations):
    timer(start_time)
    params['seed'] = seed
    #start SK fold
    for id_train, id_test in SKfold.split(X, y_train):
        #x train, x validation
        xtr, xvl = X.loc[id_train], X.loc[id_test]
        #y train, y validation
        ytr, yvl = y_train[id_train], y_train[id_test]
        #efficient datastructures for lgbm
        dtrain = lgbm.Dataset(data=xtr, label=ytr)
        dval = lgbm.Dataset(data=xvl, label=yvl, reference=dtrain)
        #model training
        bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dval, feval=evalerror, verbose_eval=100,
                         early_stopping_rounds=100)
        #add best tree and fold scores to data structure
        best_trees.append(bst.best_iteration)
        fold_scores.append(bst.best_score)
        #predict for our test set with best tree from fold. Sums the probabilities
        cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
    #average the predictions for our 5 folds
    cv_pred /= folds

pd.DataFrame({'id': test_id, 'target': cv_pred / iterations}).to_csv('lgbm_pred5-with-encodingscaling.csv', index=False)

if Kaggle == False:
    test_score = gini(target_test, cv_pred / iterations)
    print("Score on the test data")
    print("Gini")
    print(test_score)


 Time taken: 0 hours 0 minutes and 0.0 seconds


C:\Users\thoma\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.151579	valid_0's gini: 0.275997
[200]	valid_0's binary_logloss: 0.1513	valid_0's gini: 0.282682
[300]	valid_0's binary_logloss: 0.151211	valid_0's gini: 0.284987
[400]	valid_0's binary_logloss: 0.151194	valid_0's gini: 0.28541
[500]	valid_0's binary_logloss: 0.151186	valid_0's gini: 0.285667
Early stopping, best iteration is:
[464]	valid_0's binary_logloss: 0.151178	valid_0's gini: 0.286068


C:\Users\thoma\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.151305	valid_0's gini: 0.285804
[200]	valid_0's binary_logloss: 0.150978	valid_0's gini: 0.293999
[300]	valid_0's binary_logloss: 0.150864	valid_0's gini: 0.297001
[400]	valid_0's binary_logloss: 0.150837	valid_0's gini: 0.297286
Early stopping, best iteration is:
[370]	valid_0's binary_logloss: 0.150827	valid_0's gini: 0.29771
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.152177	valid_0's gini: 0.25852
[200]	valid_0's binary_logloss: 0.151943	valid_0's gini: 0.265777
[300]	valid_0's binary_logloss: 0.151851	valid_0's gini: 0.269261
[400]	valid_0's binary_logloss: 0.151805	valid_0's gini: 0.270663
[500]	valid_0's binary_logloss: 0.151779	valid_0's gini: 0.271642
[600]	valid_0's binary_logloss: 0.151775	valid_0's gini: 0.271986
Did not meet early stopping. Best iteration is:
[550]	valid_0's binary_logloss: 0.151766	valid_0's gini: 0.272237
Tra

C:\Users\thoma\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.151625	valid_0's gini: 0.275666
[200]	valid_0's binary_logloss: 0.151359	valid_0's gini: 0.282139
[300]	valid_0's binary_logloss: 0.151257	valid_0's gini: 0.284361
[400]	valid_0's binary_logloss: 0.151215	valid_0's gini: 0.285315
[500]	valid_0's binary_logloss: 0.151203	valid_0's gini: 0.285903
Early stopping, best iteration is:
[481]	valid_0's binary_logloss: 0.151191	valid_0's gini: 0.286294
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.151301	valid_0's gini: 0.286941
[200]	valid_0's binary_logloss: 0.150991	valid_0's gini: 0.294536
[300]	valid_0's binary_logloss: 0.15088	valid_0's gini: 0.297241
[400]	valid_0's binary_logloss: 0.150839	valid_0's gini: 0.298296
[500]	valid_0's binary_logloss: 0.15084	valid_0's gini: 0.298272
[600]	valid_0's binary_logloss: 0.150812	valid_0's gini: 0.298999
Did not meet early stopping. Best iteration is:
[60

C:\Users\thoma\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.151587	valid_0's gini: 0.276099
[200]	valid_0's binary_logloss: 0.151324	valid_0's gini: 0.282573
[300]	valid_0's binary_logloss: 0.151206	valid_0's gini: 0.285597
[400]	valid_0's binary_logloss: 0.151178	valid_0's gini: 0.286098
[500]	valid_0's binary_logloss: 0.151191	valid_0's gini: 0.286251
Early stopping, best iteration is:
[433]	valid_0's binary_logloss: 0.151167	valid_0's gini: 0.28662
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.15133	valid_0's gini: 0.284888
[200]	valid_0's binary_logloss: 0.151023	valid_0's gini: 0.293177
[300]	valid_0's binary_logloss: 0.1509	valid_0's gini: 0.295766
[400]	valid_0's binary_logloss: 0.150838	valid_0's gini: 0.297041
[500]	valid_0's binary_logloss: 0.150822	valid_0's gini: 0.297438
[600]	valid_0's binary_logloss: 0.1508	valid_0's gini: 0.297802
Did not meet early stopping. Best iteration is:
[600]	v

Our final result on the test set is

In [11]:
"Score on the test data"
"Gini"
"0.2788912296158311"

'0.2788912296158311'

Separately, we run the same code but with the mean imputed data and our score is 0.2704817256788792